In [116]:
import sqlite3
from sqlite3 import Error
import pandas as pd

In [9]:
conn = sqlite3.connect("./Country_Club_Data/data.db")

In [10]:
cur = conn.cursor()

In [114]:
q = """SELECT facid, SUM(cost) FROM (SELECT Facilities.facid,
            CASE
                WHEN Bookings.memid = CAST(0 as integer)
                    THEN CAST(Facilities.guestcost as integer) * CAST(slots as integer)
                ELSE CAST(Facilities.membercost as integer) * CAST(slots as integer)
            END as 'cost'
            FROM Bookings
            LEFT JOIN Facilities ON Bookings.facid = Facilities.facid)
            GROUP BY facid"""
cur.execute (q)
results = cur.fetchall()
for x in results:
    print(x)

('0', 13860)
('1', 14310)
('2', 1845)
('3', 180)
('4', 49556)
('5', 14406)
('6', 12916)
('7', 240)
('8', 270)


In [117]:
df = pd.read_sql_query(q, conn)
df

,facid,SUM(cost)
0,0,13860
1,1,14310
2,2,1845
3,3,180
4,4,49556
5,5,14406
6,6,12916
7,7,240
8,8,270
